In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import my_app

my_app.my_app()


Token Valid: Alison Myers (185698)



Inter Peer Review Group 1 427331
93991 - 427331 (1680279)
Inter Peer Review Group 2 427332
93992 - 427332 (1680175)
93994 - 427332 (1680178)
93997 - 427332 (1680376)
93999 - 427332 (1680457)
TEST 1 428996
93999 - 428996 (1685944)
93994 - 428996 (1685945)
93997 - 428996 (1685946)
TEST 2 428997
93995 - 428997 (1685941)
93992 - 428997 (1685942)
93991 - 428997 (1685943)
